In [ ]:
import numpy as np
from min_regressor_f_py import min_regressor_f_py as W_min
q  = 0.3*np.ones(6)
dq = 0.6*np.ones(6)
ddq= 0.7*np.ones(6)
pi_min = 0.1 + np.arange(48) * 0.05
W  = W_min(q, dq, ddq)

np.set_printoptions(precision=4, suppress=True)

print(W @ pi_min)        # Display the regressor matrix

In [ ]:
import torch
from min_regressor_pytorch import min_regressor_pytorch as W_min
# 创建输入张量，开启自动求导
# q  = 0.3*torch.ones(6, requires_grad=True)
# dq = 0.6*torch.ones(6, requires_grad=True)
# ddq= 0.7*torch.ones(6, requires_grad=True)


# q = torch.tensor([0.3]*6, requires_grad=True)
# dq = torch.tensor([0.6]*6, requires_grad=True)
# ddq = torch.tensor([0.7]*6, requires_grad=True)

q = torch.full((6,), 0.3, requires_grad=True)  # 直接创建叶子张量
dq = torch.full((6,), 0.6, requires_grad=True)  # 直接创建叶子张量
ddq = torch.full((6,), 0.7, requires_grad=True)  # 直接创建叶子张量

pi_min = 0.1 + torch.arange(48) * 0.05
W  = W_min(q, dq, ddq)


output = W @ pi_min  # 矩阵乘法，结果是 (6, )

print(output)
print(W.requires_grad)

# 测试是否能求梯度
output_sum = output.sum()
output_sum.backward()

print(q.grad)  # 如果非None，说明梯度支持正常
print(dq.grad)
print(ddq.grad)   
print("q.is_leaf:", q.is_leaf)
print("dq.is_leaf:", dq.is_leaf)
print("ddq.is_leaf:", ddq.is_leaf)

In [ ]:
import torch
from min_regressor_pytorch_matrix import min_regressor_pytorch_matrix as W_min_matrix
# 创建输入张量，开启自动求导

q = torch.full((10,6), 0.3, requires_grad=True)  # 直接创建叶子张量
dq = torch.full((10,6), 0.6, requires_grad=True)  # 直接创建叶子张量
ddq = torch.full((10,6), 0.7, requires_grad=True)  # 直接创建叶子张量

pi_min = 0.1 + torch.arange(48) * 0.05
W  = W_min_matrix(q, dq, ddq)


output = W @ pi_min  # 矩阵乘法，结果是 (6, )

print(output)
print(W.requires_grad)
print(W.shape)
# 测试是否能求梯度
output_sum = output.sum()
output_sum.backward()

# print(q.grad)  # 如果非None，说明梯度支持正常
# print(dq.grad)
# print(ddq.grad)   
# print("q.is_leaf:", q.is_leaf)
# print("dq.is_leaf:", dq.is_leaf)
# print("ddq.is_leaf:", ddq.is_leaf)

In [1]:
import torch

import time
from min_regressor_pytorch_matrix import min_regressor_pytorch_matrix as W_min_matrix
from min_regressor_pytorch_matrix_pro import min_regressor_pytorch_matrix as W_min_matrix_pro

batch =100
device = 'cuda' 
# device = 'cpu'
q = torch.full((batch,6), 0.3, device=device, requires_grad=True)
dq = torch.full((batch,6), 0.6, device=device, requires_grad=True)
ddq = torch.full((batch,6), 0.7, device=device, requires_grad=True)
pi_min = (0.1 + torch.arange(48, device=device) * 0.05)


# W  = W_min_matrix(q, dq, ddq)
# W_p  = W_min_matrix_pro(q, dq, ddq)

# output = W @ pi_min  # 矩阵乘法，结果是 (6, )
# output = W_p @ pi_min  # 矩阵乘法，结果是 (6, )

# W_min_matrix_pro = torch.jit.trace(W_min_matrix_pro, (q, dq, ddq))



# 预热一次，避免第一次运行慢的影响
_ = W_min_matrix(q, dq, ddq) @ pi_min
_ = W_min_matrix_pro(q, dq, ddq) @ pi_min



def measure_forward_time(func, q, dq, ddq, pi_min, repeats=100):
    # torch.cuda.synchronize() if torch.cuda.is_available() else None
    start = time.time()
    for _ in range(repeats):
        W = func(q, dq, ddq)
        out = W @ pi_min
    # torch.cuda.synchronize() if torch.cuda.is_available() else None
    end = time.time()
    avg_time_ms = (end - start) * 1000 / repeats
    return avg_time_ms

t2 = measure_forward_time(W_min_matrix_pro, q, dq, ddq, pi_min)
t1 = measure_forward_time(W_min_matrix, q, dq, ddq, pi_min)


print(f"W_min_matrix forward avg time: {t1:.4f} ms")
print(f"W_min_matrix_pro forward avg time: {t2:.4f} ms")

W_min_matrix forward avg time: 15.2809 ms
W_min_matrix_pro forward avg time: 14.2800 ms


In [ ]:
import torch
import time
from min_regressor_pytorch_matrix import min_regressor_pytorch_matrix as W_min_matrix
from min_regressor_pytorch_matrix_pro import min_regressor_pytorch_matrix as W_min_matrix_pro

def run_test(device: str):
    print(f"\nRunning on device: {device}")

    batch = 10
    # 创建输入张量并放到指定设备，requires_grad=True 保持不变
    q = torch.full((batch, 6), 0.3, device=device, requires_grad=True)
    dq = torch.full((batch, 6), 0.6, device=device, requires_grad=True)
    ddq = torch.full((batch, 6), 0.7, device=device, requires_grad=True)

    pi_min = (0.1 + torch.arange(48, device=device) * 0.05).float()

    # JIT trace pro 版本（只跟踪一次）
    W_min_matrix_pro_jit = torch.jit.trace(W_min_matrix_pro, (q, dq, ddq))

    # 预热，防止第一次慢
    _ = W_min_matrix(q, dq, ddq) @ pi_min
    _ = W_min_matrix_pro_jit(q, dq, ddq) @ pi_min

    def measure_forward_time(func, q, dq, ddq, pi_min, repeats=100):
        if device == 'cuda':
            torch.cuda.synchronize()
        start = time.time()
        for _ in range(repeats):
            W = func(q, dq, ddq)
            out = W @ pi_min
        if device == 'cuda':
            torch.cuda.synchronize()
        end = time.time()
        avg_time_ms = (end - start) * 1000 / repeats
        return avg_time_ms
    t2 = measure_forward_time(W_min_matrix_pro_jit, q, dq, ddq, pi_min)

    t1 = measure_forward_time(W_min_matrix, q, dq, ddq, pi_min)

    print(f"W_min_matrix forward avg time: {t1:.4f} ms")
    print(f"W_min_matrix_pro_jit forward avg time: {t2:.4f} ms")

if __name__ == "__main__":
    # 测试CPU
    # run_test("cpu")

    # # 如果有GPU，测试GPU
    if torch.cuda.is_available():
        run_test("cuda")
    else:
        print("\nCUDA is not available on this machine.")
